In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import time
import pandas as pd


In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

In [12]:
def create_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model


In [13]:
def benchmark(device_name):
    with tf.device(device_name):
        model = create_model()
        start_time = time.time()
        model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=0)
        training_time = time.time() - start_time
        _, accuracy = model.evaluate(x_test, y_test, verbose=0)
        return training_time, accuracy

# Run benchmarks
print("=== GPU vs CPU Benchmark ===")
print("Running CPU benchmark...")
cpu_time, cpu_acc = benchmark('/CPU:0')
print(f"CPU Time: {cpu_time:.2f}s | Accuracy: {cpu_acc:.4f}")

if tf.config.list_physical_devices('GPU'):
    print("\nRunning GPU benchmark...")
    gpu_time, gpu_acc = benchmark('/GPU:0')
    print(f"GPU Time: {gpu_time:.2f}s | Accuracy: {gpu_acc:.4f}")
    print(f"\nSpeedup Factor: {cpu_time/gpu_time:.1f}x")
else:
    print("\nNo GPU detected! Check CUDA/cuDNN installation.")

=== GPU vs CPU Benchmark ===
Running CPU benchmark...




CPU Time: 26.16s | Accuracy: 0.9767

No GPU detected! Check CUDA/cuDNN installation.


In [14]:
print("Starting benchmarks...")
print(f"Available GPUs: {tf.config.list_physical_devices('GPU')}\n")

print("Running CPU benchmark...")
cpu_time, cpu_acc = benchmark('/CPU:0')


Starting benchmarks...
Available GPUs: []

Running CPU benchmark...


In [15]:
gpu_time, gpu_acc = None, None
if tf.config.list_physical_devices('GPU'):
    print("\nRunning GPU benchmark...")
    gpu_time, gpu_acc = benchmark('/GPU:0')
else:
    print("\nNo GPU available, skipping GPU benchmark")



No GPU available, skipping GPU benchmark


In [16]:
results = pd.DataFrame({
    'Device': ['CPU', 'GPU'],
    'Training Time (s)': [cpu_time, gpu_time],
    'Test Accuracy': [cpu_acc, gpu_acc],
    'Speedup Factor': [1.0, cpu_time/gpu_time if gpu_time else None]
})

print("\nBenchmark Results:")
print(results.to_string(index=False))



Benchmark Results:
Device  Training Time (s)  Test Accuracy  Speedup Factor
   CPU          24.600861         0.9757             1.0
   GPU                NaN            NaN             NaN


In [17]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow Version: 2.15.0
GPU Available: []
